# Account


In [16]:
# | default_exp classes.DomoAccount

In [17]:
# | export

from enum import Enum
from dataclasses import dataclass, field
from abc import ABC, abstractclassmethod, abstractmethod

import datetime as dt
import re


import aiohttp

from fastcore.basics import patch_to

import domolibrary.utils.convert as cd
import domolibrary.utils.DictDot as util_dd
import domolibrary.client.DomoAuth as dmda
import domolibrary.routes.account as account_routes

# Account Connector Config

Each Domo Dataset that pulls data into Vault must have a stream, which stores the configuration information related to which data is exctracted from a datasource. Each stream has an associated account which stores the source data's authentication information.

Because each datasource may have different authentication parameters, there may be multiple versions of the same account type (with different credentials) or multiple account types deployed within a domo instance if the user is extracting data from multiple systems.

Account's can be configured such that certain fields are designated as encrypted fields, and the user will never be able to see the encrypted values EXCEPT in Domo's Jupyter Notebook integration.


In [18]:
class DomoAccount_Config(ABC):
    """DomoAccount Config abstract base class"""

    @abstractclassmethod
    def _from_json(cls, obj):
        """convert accounts API response into a class object"""
        pass

    @abstractmethod
    def to_json(self):
        """convert class object into a format the accounts API expects"""
        pass

In [19]:
# | export
@dataclass
class DomoAccount_Config_AbstractCredential(DomoAccount_Config):
    credentials: dict

    def __post_init_(self):
        super().__init__(self)

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            credentials=dd.credentials,
        )

    def to_json(self):
        return {"credentials": self.credentials}

In [20]:
# | export
@dataclass
class DomoAccount_Config_DatasetCopy(DomoAccount_Config):
    domo_instance: str
    access_token: str = field(repr=False)
    

    def __post_init_(self):
        super().__init__(self)

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            access_token=dd.accessToken,
            domo_instance=dd.instance
        )

    def to_json(self):
        return {"accessToken": self.access_token,
                "instance": self.domo_instance
                }


In [21]:
# | export
@dataclass
class DomoAccount_Config_Governance(DomoAccount_Config):
    domo_instance: str
    access_token: str = field(repr=False)

    def __post_init_(self):
        super().__init__(self)

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            access_token=dd.apikey,
            domo_instance=dd.customer
        )

    def to_json(self):
        return {"apikey": self.access_token,
                "customer": self.domo_instance
                }


In [22]:
# | export
@dataclass
class DomoAccount_Config_HighBandwidthConnector(DomoAccount_Config):
    aws_access_key: str
    aws_secret_key: str = field(repr=False)
    s3_staging_dir: str
    region: str = "us-west-2"
    data_provider_type = "amazon-athena-high-bandwidth"

    def __post_init_(self):
        super().__init__(self)

    @classmethod
    def _from_json(cls, obj):

        dd = util_dd.DictDot(obj)

        return cls(
            aws_access_key=dd.awsAccessKey,
            aws_secret_key=dd.awsSecretKey,
            s3_staging_dir=dd.s3StagingDir,
            region=dd.region,
        )

    def to_json(self):
        return {
            "awsAccessKey": self.aws_access_key,
            "awsSecretKey": self.aws_secret_key,
            "s3StagingDir": self.s3_staging_dir,
            "region": self.region,
        }

In [23]:
# | export
class AccountConfig(Enum):
    """
    Enum provides appropriate spelling for data_provider_type and config object.  
    The name of the enum should correspond with the data_provider_type with hyphens replaced with underscores.
    """

    amazon_athena_high_bandwidth = DomoAccount_Config_HighBandwidthConnector

    abstract_credential_store = DomoAccount_Config_AbstractCredential

    dataset_copy = DomoAccount_Config_DatasetCopy

    domo_governance_d14c2fef_49a8_4898_8ddd_f64998005600 = DomoAccount_Config_Governance


In [24]:
# | export
@dataclass
class DomoAccount:
    name: str
    data_provider_type: str

    id: int = None
    created_dt: dt.datetime = None
    modified_dt: dt.datetime = None
    auth: dmda.DomoAuth = field(repr=False, default=None)

    config: DomoAccount_Config = None

    @classmethod
    def _from_json(cls, obj: dict, auth: dmda.DomoAuth = None):
        """converts data_v1_accounts API response into an accounts class object"""

        dd = util_dd.DictDot(obj)

        return cls(
            id=dd.id,
            name=dd.displayName,
            data_provider_type=dd.dataProviderType,
            created_dt=cd.convert_epoch_millisecond_to_datetime(dd.createdAt),
            modified_dt=cd.convert_epoch_millisecond_to_datetime(dd.modifiedAt),
            auth=auth,
        )
    
    async def _get_config(self, session = None, debug_api : bool = None, return_raw: bool = False):
    
        res_config = await account_routes.get_account_config(
            auth=self.auth,
            account_id=self.id,
            data_provider_type=self.data_provider_type,
            session=session,
            debug_api=debug_api
        )

        if return_raw:
            return res_config

        enum_clean = re.sub("-", "_", self.data_provider_type)

        if not enum_clean in AccountConfig.__members__:
            return None

        self.config = (
            AccountConfig[enum_clean].value._from_json(res_config.response)
        )

        return self.config


In [25]:
@patch_to(DomoAccount, cls_method=True)
async def get_from_id(
    cls,
    auth: dmda.DomoAuth,
    account_id: int,
    session: aiohttp.ClientSession = None,
    return_raw: bool = False,
    debug_api: bool = False
):

    res = await account_routes.get_account_from_id(
        auth=auth, account_id=account_id, session=session, debug_api = debug_api
    )
    if return_raw:
        return res

    if not res.is_success:
        return None

    obj = res.response
    acc = cls._from_json(obj, auth)

    await acc._get_config(session = session, debug_api = debug_api)

    return acc

#### sample implementation of get_from_id


In [26]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

await DomoAccount.get_from_id(auth=token_auth, account_id=5, return_raw=False)

DomoAccount(name='test_rename_using_classmethod', data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 1, 4, 22, 53, 30), config=DomoAccount_Config_Governance(domo_instance='domo-dojo'))

In [34]:
@patch_to(DomoAccount)
async def update_config(self: DomoAccount,
                        auth: dmda.DomoAuth = None,
                        debug_api: bool = False,
                        config : DomoAccount_Config = None,
                        session: aiohttp.ClientSession = None, return_raw: bool = False):

    auth = auth or self.auth

    config = config or self.config

    res = await account_routes.update_account_config(auth=auth,
                                                     account_id=self.id,
                                                     data_provider_type=self.data_provider_type,
                                                     config_body=config.to_json(),
                                                     debug_api=debug_api, session=session)

    if return_raw:
        return res

    await self._get_config(session=session, debug_api=debug_api)

    return self


#### Sample implementation of update_config

In [41]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

domo_instance = 'domo-dojo'
access_token = os.environ['DOMO_DOJO_ACCESS_TOKEN']

# creates a DomoAccount object
domo_account = await DomoAccount.get_from_id(auth=token_auth, account_id=5)


# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.config.domo_instance = 'domo-dojo'
domo_account.config.access_token = os.environ['DOMO_DOJO_ACCESS_TOKEN']
await domo_account.update_config()

# update domo Account API by passing new config object
config = AccountConfig.domo_governance_d14c2fef_49a8_4898_8ddd_f64998005600.value(domo_instance=domo_instance,
                                                                                  access_token=access_token)
await domo_account.update_config(config = config)


DomoAccount(name='test_rename_using_classmethod', data_provider_type='domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', id=5, created_dt=datetime.datetime(2021, 3, 26, 16, 54, 41), modified_dt=datetime.datetime(2023, 1, 4, 22, 53, 30), config=DomoAccount_Config_Governance(domo_instance='domo-dojo'))

In [45]:
class DomoAccount_UpdateName_Error(Exception):
    pass

@patch_to(DomoAccount)
async def update_name(self: DomoAccount,
                      account_name: str = None,
                      auth: dmda.DomoAuth = None,
                      debug_api: bool = False,
                      session: aiohttp.ClientSession = None,
                      return_raw: bool = False):

    auth = auth or self.auth

    # print(auth, self.id, self.data_provider_type, self.config.to_json())

    res = await account_routes.update_account_name(auth=auth,
                                                   account_id=self.id,
                                                   account_name=account_name or self.name,
                                                   debug_api=debug_api,
                                                   session=session)

    if return_raw:
        return res

    if not res.is_success:
        raise DomoAccount_UpdateName_Error()
    
    self = await self.get_from_id(auth=auth, account_id=self.id)
    
    return self


#### sample implementation of update_name

In [48]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-dojo", domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"]
)

# creates a DomoAccount object
domo_account = await DomoAccount.get_from_id(auth=token_auth, account_id=5)

account_name = 'test_rename_using_method'

# update domo Account API without passing explicit config object
# adjust configuration information for that object
domo_account.name = account_name
await domo_account.update_name()

# update domo Account API by passing account_name str
await domo_account.update_name(account_name= account_name, return_raw=True)


ResponseGetData(status=200, response={'id': 5, 'userId': 1893952720, 'name': 'Domo Governance Datasets Third Party Account', 'displayName': 'test_rename_using_method', 'type': 'data', 'valid': True, 'dataProviderType': 'domo-governance-d14c2fef-49a8-4898-8ddd-f64998005600', 'credentialsType': 'fields', 'createdAt': 1616777681000, 'createdBy': 1893952720, 'modifiedAt': 1674525525000, 'modifiedBy': 1893952720, 'configurations': {'apikey': '{encrypted}NjcyMzdmMGQtZjBhMy00NjJhLTllZGItMzM2ZjE0MDkxMjFkynNPd1fe6uzpUgZQRWLDbLeG+r5DAaZfvVQWSPTEQAoWxuj5P8nCUpsH4+NyniuGtaV95aHvB9E+DzbP2UUhuDjjiPVIWTDpUv3ww++vNXY=', 'customer': '{encrypted}OTQzM2IzMDMtYTMyMy00OTE2LThlNGQtNzJiZjJlNzZiMDhh0RJg3wKTrKSMH22WaRbUw92pVMKS3WqaI1FcJBdz1j4='}, 'accountTemplateId': None, 'accountTemplateAuthorizationId': None, 'accountId': 5}, is_success=True)

In [30]:


# def config_to_json(self):
#     obj = {'displayName': self.name,
#            'dataProviderType': self.data_provider_type,
#            'name': self.data_provider_type,
#            'configurations': self.config.to_json()}

#     return obj

# async def create_account(self,
#                          auth: dmda.DomoFullAuth = None,
#                          #  config_body: dict,
#                          debug: bool = False, log_results: bool = False, session: aiohttp.ClientSession = None):
#     auth = auth or self.auth

#     res = await account_routes.create_account(auth=auth,
#                                               config_body=self.config_to_json(),
#                                               debug=debug, log_results=log_results, session=session)

#     if debug:
#         print(res)

#     if res.status != 200:
#         return False

#     obj = res.response
#     acc = await self.get_from_id(auth=auth, account_id=obj.get('id'))
#     acc.status = res.status
#     acc.is_success = res.is_success
#     return acc

# async def delete_account(self,
#                          auth: dmda.DomoFullAuth = None,
#                          #  config_body: dict,
#                          debug: bool = False, log_results: bool = False, session: aiohttp.ClientSession = None):
#     auth = auth or self.auth

#     res = await account_routes.delete_account(auth=auth,
#                                               account_id = self.id,
#                                               debug=debug, log_results=log_results, session=session)

#     if debug:
#         print(res)

#     if res.status != 200:
#         return False

#     return True